Implenntation of Transformer Arch 

<img src="image.png" width="500" height="600">



In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# from utils import utils

In [3]:
def initialize_weight(x):
    nn.init.xavier_uniform_(x.weight)
    if x.bias is not None:
        nn.init.constant_(x.bias, 0)
        
        
class FeedForwardNetwork(nn.Module):
     def __init__(self, hidden_size, filter_size, dropout_rate):
        super(FeedForwardNetwork, self).__init__()

        self.layer1 = nn.Linear(hidden_size, filter_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.layer2 = nn.Linear(filter_size, hidden_size)

        initialize_weight(self.layer1)
        initialize_weight(self.layer2)
        
     def forward(self,x):
        x= self.layer1(x)
        x= self.relu(x)
        x= self.dropout(x)
        x= self.layer(x)
        
        return x